In [ ]:
!pip install transformers pinecone-client flask


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-category-dataset")

print("Path to dataset files:", path)

100%|██████████| 26.5M/26.5M [00:01<00:00, 16.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3


In [ ]:
print(path)


/root/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3


In [ ]:
import os

# List all files in the dataset directory
files = os.listdir(path)
print("Files in dataset directory:", files)


Files in dataset directory: ['News_Category_Dataset_v3.json']


In [ ]:
import json
import os

# Load the NDJSON file
file_path = os.path.join(path, "News_Category_Dataset_v3.json")

data = []
with open(file_path, "r") as f:
    for line in f:
        data.append(json.loads(line))

# Explore the first few entries
for i, item in enumerate(data[:5]):
    print(f"Entry {i+1}: {item}")



Entry 1: {'link': 'https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9', 'headline': 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters', 'category': 'U.S. NEWS', 'short_description': 'Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.', 'authors': 'Carla K. Johnson, AP', 'date': '2022-09-23'}
Entry 2: {'link': 'https://www.huffpost.com/entry/american-airlines-passenger-banned-flight-attendant-punch-justice-department_n_632e25d3e4b0e247890329fe', 'headline': 'American Airlines Flyer Charged, Banned For Life After Punching Flight Attendant On Video', 'category': 'U.S. NEWS', 'short_description': "He was subdued by passengers and crew when he fled to the back of the aircraft after the confrontation, according to the U.S. attorney's office in Los Angeles.", 'authors': 'Mary Papenfuss', 'date': '2022-09-23'}
Entry 3: {'link': 'htt

In [ ]:
import pandas as pd

# Load the NDJSON file into a DataFrame
df = pd.read_json(file_path, lines=True)

# Display the first few rows
print(df.head())


                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

In [ ]:
import json
from sentence_transformers import SentenceTransformer
import pinecone
from flask import Flask, request, jsonify
import os

# Initialize Model and Pinecone
model = SentenceTransformer("all-MiniLM-L6-v2")

pinecone.init(api_key="YOUR_PINECONE_API_KEY", environment="us-west1-gcp")
index_name = "news-category-search"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384)
index = pinecone.Index(index_name)

# Flask App
app = Flask(__name__)

# Load and Preprocess Dataset
def load_dataset(file_path):
    documents = []
    with open(file_path, "r") as f:
        data = json.load(f)
        for i, item in enumerate(data):
            content = f"{item['headline']} - {item['short_description']}"
            documents.append({"id": str(i), "content": content})
    return documents

def embed_and_store(documents):
    texts = [doc["content"] for doc in documents]
    embeddings = model.encode(texts, convert_to_tensor=True)
    for doc, emb in zip(documents, embeddings):
        index.upsert([(doc["id"], emb.tolist())])

# Endpoint for Searching
@app.route("/search", methods=["POST"])
def search():
    query = request.json.get("query", "")
    top_k = request.json.get("top_k", 5)
    query_embedding = model.encode(query, convert_to_tensor=True)
    results = index.query(query_embedding.tolist(), top_k=top_k, include_metadata=True)
    return jsonify(results)

if __name__ == "__main__":
    # Load and Embed Dataset
    dataset_path = "News_Category_Dataset_v2.json"
    if os.path.exists(dataset_path):
        documents = load_dataset(dataset_path)
        embed_and_store(documents)
        print(f"Dataset indexed in Pinecone: {len(documents)} documents.")
    else:
        print("Dataset file not found. Please download it from Kaggle.")

    # Run the App
    app.run(debug=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

